In [94]:
import utils
import GurobiQB as qb
import eventgenerator as eg
import matchers
from offline_solver import solve_offline_optimal_with_markov

In [95]:

# Parameters
num_nodes = 5
skip_prob = 0  
extra_edges = 0  
simulation_time = 20

rate_riders = 0.4  
rate_drivers = 0.3  
sojourn_rate_riders = 0.2 
sojourn_rate_drivers = 0.5
reward_value = num_nodes + 2
distance_penalty = 1

# Generate the adjacency matrix and reward matrix
adj_matrix = utils.generate_imperfect_grid_adjacency_matrix(num_nodes, skip_prob, extra_edges)
rewards = utils.adjacency_to_rewards(adj_matrix, reward_value, distance_penalty)

# Define arrival rates for active and passive types based on nodes
lambda_i = {f"Active Driver Node {i}": rate_drivers for i in range(num_nodes)}
lambda_j = {f"Passive Rider Node {i}": rate_riders for i in range(num_nodes)}

# Define abandonment rates for active types (example values)
mu_i = {f"Active Driver Node {i}": sojourn_rate_drivers for i in range(num_nodes)}

In [96]:

# Obtain full results from QB optimization
QB_results = qb.solve_QB(rewards, lambda_i, lambda_j, mu_i)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 40 columns and 105 nonzeros
Model fingerprint: 0x12773d11
Model has 25 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [5e-01, 5e-01]
  Objective range  [3e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 4e-01]
Presolve removed 40 rows and 5 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 40 rows and 5 columns
Presolve time: 0.00s
Presolved: 335 rows, 186 columns, 835 nonzeros
Presolved model has 150 bilinear constraint(s)
Variable types: 186 continuous, 0 integer (0 binary)
Found heuristic solution: objective 6.6760336

Root relaxation: objective 8.342108e+00, 167 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node 

In [97]:
event_queue = eg.EventQueue()
eg.generate_events(event_queue, rate_riders, rate_drivers, sojourn_rate_riders, sojourn_rate_drivers, num_nodes, simulation_time)

print("EVENT QUEUE\n")
event_queue.print_queue()
print("\n\n\n")

# Clone the event queue for multiple simulations
event_queue_2 = event_queue.clone()
event_queue_3 = event_queue.clone()
event_queue_4 = event_queue.clone()
event_queue_5 = event_queue.clone()

EVENT QUEUE

Current Event Queue:
Time: 0.198549, Event Type: arrival, Entity: [Type: Rider, Location: 4, Arrival: 0.198549]
Time: 0.229295, Event Type: arrival, Entity: [Type: Driver, Location: 1, Arrival: 0.229295]
Time: 0.285525, Event Type: arrival, Entity: [Type: Driver, Location: 0, Arrival: 0.285525]
Time: 0.311449, Event Type: arrival, Entity: [Type: Rider, Location: 4, Arrival: 0.311449]
Time: 0.449757, Event Type: abandonment, Entity: [Type: Driver, Location: 1, Arrival: 0.229295]
Time: 0.904692, Event Type: abandonment, Entity: [Type: Rider, Location: 4, Arrival: 0.311449]
Time: 1.023348, Event Type: arrival, Entity: [Type: Driver, Location: 3, Arrival: 1.023348]
Time: 1.199758, Event Type: arrival, Entity: [Type: Rider, Location: 1, Arrival: 1.199758]
Time: 1.415102, Event Type: arrival, Entity: [Type: Rider, Location: 3, Arrival: 1.415102]
Time: 1.633238, Event Type: arrival, Entity: [Type: Rider, Location: 0, Arrival: 1.633238]
Time: 1.685027, Event Type: arrival, Entity:

In [98]:

print("GREEDY SOLUTION\n")
matchers.greedy_matcher(event_queue_5, rewards, QB_results, lambda_i, lambda_j, mu_i, adj_matrix)

GREEDY SOLUTION

Processing event: arrival for entity: <eventgenerator.Rider object at 0x10f49f140>
Rider arrived at location 4 at time 0.198549.
Processing event: arrival for entity: <eventgenerator.Driver object at 0x10f485d60>
Driver arrived at location 1 at time 0.229295.
Processing event: arrival for entity: <eventgenerator.Driver object at 0x10efd79e0>
Driver arrived at location 0 at time 0.285525.
Processing event: arrival for entity: <eventgenerator.Rider object at 0x10f49ca70>
Rider arrived at location 4 at time 0.311449.
Attempting to match Driver at 4 matched with Rider at 1.
Rider at 4 matched with Driver at 1.
Processing event: arrival for entity: <eventgenerator.Driver object at 0x10f0155e0>
Driver arrived at location 3 at time 1.023348.
Processing event: arrival for entity: <eventgenerator.Rider object at 0x10f485970>
Rider arrived at location 1 at time 1.199758.
Attempting to match Driver at 1 matched with Rider at 0.
Rider at 1 matched with Driver at 0.
Processing even

In [99]:
print("\nOFFLINE OPTIMAL SOLUTION\n")
solve_offline_optimal_with_markov(event_queue_4, rewards)


OFFLINE OPTIMAL SOLUTION


Summary Statistics:
Number of Drivers Matched: 73
Number of Riders Matched: 73
Total Riders Processed: 76
Total Drivers Processed: 73
Total Rewards: 498.00
Average Reward per Transaction: 6.82
Average Trip Distance: 0.18 units
Average Wait Time: 1.54 units
